In [ ]:
#https://arxiv.org/pdf/1512.03385.pdf
#https://github.com/njainds/keras/blob/master/examples/cifar10_resnet.py

In [2]:
import numpy as np
import keras
from keras.layers import Dense,Input,Conv2D, Activation,BatchNormalization,AveragePooling2D,Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.regularizers import l2
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input
#from kt_utils import * 
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline
import pandas as pd
import PIL
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [3]:
def read_data(path):
    train=pd.read_csv(path+'/train.csv')
    test=pd.read_csv(path+'/test.csv')
    return train,test

train,test=read_data('./Datasets')

def generate_input(path,image_ids):
    for id in image_ids:
        img=image.load_img(path+id,target_size=(32,32))
        x=image.img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        yield x
    
X=np.concatenate([feat for feat in generate_input('./Datasets/Train/',train['ID'])])
#X_test=np.concatenate([feat for feat in generate_input('./Datasets/Test/',test['ID'])])

lb = preprocessing.LabelBinarizer()
y=lb.fit_transform(train['Class'])
#np_utils.to_categorical(train['Class'], 3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
#Define model parameters
batch_size=128
epochs=200
depth=32
version=1
data_augmentation=False
num_classes=3
modeltype='Resnet%dv%d' % (depth,version)

In [6]:
def lr_schedule(epoch):
    lr=1e-3
    if epoch>180:
        lr*=0.5e-3
    elif epoch>160:
        lr*=1e-3
    elif epoch>120:
        lr*=1e-2
    elif epoch>80:
        lr*=1e-1
    print('learning rate: ', lr)
    return lr

def resnet_layer(inputs,num_filters=16,kernel_size=3,strides=1,activation='relu',batch_normalization=True,
                 conv_first=True,pad='same'):
    conv=Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding=pad,
               kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))
    x=inputs
    x=conv(x)
    if batch_normalization:
        x=BatchNormalization()(x)
    if Activation is not None:
        x=Activation(activation)(x)
    return x    

In [17]:
def resnet_v1(input_shape,depth,num_classes=3):
    """
    Input->Resnet_layer->stack0->stack1->stack2->flatten->dense(3)
    Resnet_layer= conv[16,3*3]->BN->Relu
    Each stack has 6 blocks: B0,B1,B2,B3,B4,B5
    Each Block B=  Resnet_layer->Resnet_layer(no activation)-> Add with block input->Activation(relu)
    Each block within a stack has same dimensions of filters with stride=1
    dimensions change as the stack changes: 
    B0 of stack1: filters=32, stride=2 to downsample
    B0 of stack2: filters=64, stride=2 to downsample
    When processing B0 and stack2 and stack3, input_shape from B5 output of previous stack is mapped
    to B0 otput of current stack to match dimesions
    conv1  : 64x64, 16
    stage 0: 64x64, 16
    stage 1: 32x32, 32
    stage 2: 16x16, 64 
    """
    num_filters=16
    num_res_blocks=int((depth-2)/6)
    inputs=Input(shape=input_shape)
    x=resnet_layer(inputs=inputs)
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides=1
            if stack>0 and res_block==0:
                strides=2 #to downsample
            y=resnet_layer(inputs=x,num_filters=num_filters,strides=strides)
            y=resnet_layer(inputs=y,num_filters=num_filters,activation=None)
            if stack>0 and res_block==0:
                #change dimensions of inputs as the dimesnions in new stack is different (Padding by zeros)
                x=resnet_layer(inputs=x,num_filters=num_filters,kernel_size=1,strides=strides,activation=None,batch_normalization=False)
            x=keras.layers.add([x,y])
            x=Activation('relu')(x)
        num_filters*=2
    #This resnet doesn't have BN at end of res blocks
    x=AveragePooling2D(pool_size=8)(x)
    y=Flatten()(x)
    outputs=Dense(num_classes,activation='softmax',kernel_initializer='he_normal')(y)
    model=Model(inputs=inputs,outputs=outputs)
    return model
       

In [18]:
model=resnet_v1(input_shape=X_train.shape[1:],depth=32)
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=lr_schedule(0)),metrics=['accuracy'])
model.summary()

learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_32 (BatchNo (None, 32, 32, 16)   64          conv2d_34[0][0]                  
__________________________________________________________________________________________________
activation_49 (Activation)      (None, 32, 32, 16)   0           batch_normalization_32[0][0]     
_______________________________________________________________________________________

In [19]:
#create checkpoints
checkpoint = ModelCheckpoint(filepath='./Age_detection_resnet32.h5',monitor='val_acc',verbose=1,save_best_only=True)
lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=5,min_lr=0.5e-6)
callbacks=[checkpoint,lr_reducer,lr_scheduler]
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True,
              callbacks=callbacks)

Not using data augmentation.
Train on 13337 samples, validate on 6569 samples
Epoch 1/200
learning rate:  0.001
  384/13337 [..............................] - ETA: 3:09:14 - loss: 1.4870 - acc: 0.3698

KeyboardInterrupt: 